# PROYECTO FINAL DE CIENCIA DE DATOS - BOOTCAMP CÓDIGO FACILITO

## Realizado por Gonzalo Mahserdjian y Simón Maquilón

### INSTALAR REQUERIMIENTOS

Instalo los requerimientos necesarios para este Jupiter Notebook:

In [ ]:
!python -m pip uninstall python-dotenv pyodbc polars pandas numpy altair vega_datasets pyarrow matplotlib

In [ ]:
!python -m pip install python-dotenv pyodbc polars pandas numpy altair vega_datasets pyarrow matplotlib

In [ ]:
!pip freeze > requirements.txt

#### Imports

In [ ]:
import csv
import pyodbc
import asyncio
import polars as pl
import pandas as pd
import pyarrow as pa
import altair as alt
import matplotlib.pyplot as plt

from pathlib import Path
from vega_datasets import data
from IPython.display import display
from dotenv import load_dotenv, dotenv_values

### OBTENCIÓN DE DATOS PARA DATASETS

Para armar el DataSet con los datos con los que voy a trabajar, realicé una gran consulta de SQL (Big Query), como traer todos los años juntos a veces se colgaba la conexión por VPN o traía menos datos, los separé primero por años y luego los unifiqué en un único arhivo:

In [ ]:
load_dotenv(dotenv_path=".env", verbose=True)
cfg = dotenv_values(".env")

DATASET_SYNC = False
PRINT_QUERY = False
START_YEAR = 2018
END_YEAR = 2023
DATA_PATH = 'data/'
SQL_FILE_PATH = 'sql/bigquery.sql'

def read_sql_file(path: str, search: str, replace: str) -> None:
    """
    Edits file content
    """
    if Path(path).suffix == '.sql':
        try:
            with open(path, 'r', encoding='utf-8') as file:
                data = file.read()

            return data.replace(search, replace)

        except FileNotFoundError as error:
            print(error)
            return None

def get_query(year: str) -> str:
    return read_sql_file(SQL_FILE_PATH, '{year}', year)


if __name__ == '__main__':

    try:
        db_string = f"""DRIVER={cfg['DB_DRIVER']};
                        SERVER={cfg['DB_HOST']};
                        DATABASE={cfg['DB_DATABASE']};
                        UID={cfg['DB_USER']};
                        PWD={cfg['DB_PASSWORD']}"""

        if DATASET_SYNC:
            connect = pyodbc.connect(db_string)

            for year in range(START_YEAR, END_YEAR+1):
                if(PRINT_QUERY):
                    print(get_query(str(year)))

                with connect.cursor() as cursor:
                    cursor.execute(get_query(str(year)))
                    rows = cursor.fetchall()

                    with open(r'./'+DATA_PATH+'/DataSet_'+str(year)+'.csv', 'w', encoding="utf-8", newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow([x[0] for x in cursor.description])  # column headers

                        for row in rows:
                            writer.writerow(row)

            await asyncio.sleep(10)

            df = pl.read_csv(f'{DATA_PATH}DataSet_*.csv')
            df.write_csv(f'{DATA_PATH}DataSet.csv')

            print('>>> ¡Dataset sincronizado con éxito!')
        else:
            print('>>> ¡Sincronización de Dataset no habilitada, se utilizará la versión offline!')

    except pyodbc.OperationalError as err:
        print('>>> ¡No fue posible realizar la conexión!')
        print(err)


### LECTURA DEL CSV Y DEPURACIÓN DEL DATASET

Variables para el DataSet unificado con los últimos 6 años y para el DataSet limpio:

In [ ]:
DATA_PATH = 'data/'
file_path = f'{DATA_PATH}DataSet.csv'
file_path_cleaned = f'{DATA_PATH}DataSetCleaned.csv'

Realizo la lectura del DataSet unificado, y visualizo las primeras 5 columnas:

In [ ]:
df = pl.read_csv(file_path)
df.head()

Exploro las las métricas básicas sobre las columnas del DataSet:

In [ ]:
df.describe()

Dada la característica del DataSet que armé y a donde apunto con los mismos mucho no me sirven estos resultados, por empezar las primeras 2 columnas son IDs de clientes y sus contratos, tengo strings de nombres de barrios y constantes de nombres de planes, periodos, etc, lo único rescatable sería que "1 de Mayo" es el barrio que menor cantidad de veces sale y su opuesto "Villa Eucaristica" es el que más sale, y por otro lado que 6mbps es el plan más chico (en 2018) de internet (es un plan viejo, actualmente 50mbps es el menor) y 300mbps es el mayor, más adelante voy a comprobar todo esto.

Verifico la cantidad de líneas y columnas del DataSet:

In [ ]:
df.shape

A continuación verifico si hay valores NA provenientes desde la consulta SQL:

In [ ]:
df.select(
    pl
    .lit('NA').str
    .contains(
        pl.col('Internet_Tipo')
    )
    .filter(
        pl.col("Internet_Tipo") == True
    )
)

In [ ]:
df.select(
    pl
    .lit('NA').str
    .contains(
        pl.col('Estado')
    )
    .filter(
        pl.col("Estado") == True
    )
)

Busco si hay valores nulos:

In [ ]:
df.select(
    pl
    .all()
    .null_count()
).to_dicts()[0]

Se encontró un único valor nulo en columna "Barrio", realizo la limpieza...

In [ ]:
df = df.fill_null(value='Sin barrio')

Busco duplicados en la columna de "ID_Cliente":

In [ ]:
df.filter(
    pl
    .count("ID_Cliente")
    .over(df.columns) > 1
)

Unifico los duplicados de "ID_Cliente" manteniendo los primeros resultados de nada caso, como la consulta SQL ordena los más recientes primero, quedarían los contratos vigentes:

In [ ]:
df = df.unique(keep='first', maintain_order=True)

Realizo la misma búsqueda de duplicados en la columna de "ID_Contrato":

In [ ]:
df.filter(
    pl
    .count("ID_Contrato")
    .over(df.columns) > 1
)

Unifico ahora los duplicados de "ID Contrato":

In [ ]:
df = df.unique(keep='first', maintain_order=True)

Con el DataSet limpio, guardo todo en el archivo "DataSetCleaned.csv":

In [ ]:
df.write_csv(file_path_cleaned)

Leo el nuevo DataSet y vuelvo a verificar si quedó algún nulo y duplicados:

In [ ]:
df = pl.read_csv(file_path_cleaned)

df.select(
    pl
    .all()
    .null_count()
)

In [ ]:
df.select(
    pl
    .all()
    .null_count()
).to_dicts()[0]

In [ ]:
df.filter(
    pl
    .count("ID_Contrato")
    .over(df.columns) > 1
)

Visualizo parcialmente el DataSet:

In [ ]:
display(df)

### ANÁLISIS DE LOS DATOS OBTENIDOS

Luego de realizada la consulta con la big query al motor de MSSQL, y posterior limpieza de los datos con Polars completando 1 solo dato nulo de un barrio inexistente en la celda "Barrio" del cliente en cuestión y eliminando los duplicados a favor del más reciente, se verifica que los datos son consistentes poorque todas las lineas (rows) tienen la misma cantidad de valores en las columnas (cols).

¿Cúal es el por qué de los duplicados?: La base de datos es de un sistema donde no estaba pensado para evaluar los upgrades de los planes de internet, entonces logré obtener ese dato comparando tablas con contratos que tengan algún ticket con leyenda "Upgrade" (mejora en el plan del servicio de internet) en el descriptor del mismo, con un margen de 12 meses y esto en algun que otro cliente me trajo esos duplicados. Para generalizar, es porque en algunos sufrieron mudanzas, otros se arrepintieron del servicio o bien le ofrecieron pasarse a fibra óptica y luego por imposibilidades técnicas u otras que no puedo comentar, aún esa zona no contará con tendido de fibra óptica, dejando el ticket en "Time out" y devolviendo al cliente solamente con el servicio de Aire o Inalámbrico.

Como el gran trabajo se realizó en la big query y luego la limpieza, no se detectan anomalías. Bueno, creo que si podría citar una anomalía que me encontre al realizar la big query, en donde habían clientes que no entraban en la categoría de "Upgrade" pero no eran ni "Altas" ni "Bajas" del servicio, entonces los bauticé como "Fail Upgrade", porque fueron casos donde imposibilidades técnicas hicieron que no se concrete el Upgrade pero que no se perdía el cliente, quedaba con el mismo plan ya existente y sin cambios, o sea tampoco se subscribió al servicio de TV por IP.

A continuación se evaluarán los datos para identificar tendencias evidentes, y responder a las siguientes preguntas:
* Altas del servicio de internet SIPA-AIRE vs SIPA-FIBRA según el tipo de internet en los últimos 6 años.
* Bajas del servicio de internet SIPA-AIRE vs SIPA-FIBRA según el tipo de internet en los últimos 6 años.
* Actualizaciones del servicio de internet SIPA AIRE a SIPA-FIBRA.
* Actualizaciones del servicio de internet SIPA AIRE al mismo tipo de servicio.
* Ver que barrio es el que tiene mayor demanda en el tiempo y según el tipo de internet.
* Ver que ancho de banda es el que tiene mayor demanda en el tiempo y según el tipo de internet.
* Ver que plan de internet es el que tiene mayor demanda en el tiempo y según el tipo de internet.
* Ver que cantidad de clientes tienen contratado IPTV en el tiempo.

##### -> Altas del servicio de internet SIPA-AIRE vs SIPA-FIBRA en los últimos 6 años:

Con la siguiente fórmula consigo saber cuantos clientes de cada tipo de servicio hay en altas por año...

In [ ]:
altas = \
(
    df
    .filter(
        pl.col("Estado") == "ALTA"
    )
    .pivot(
        index = "Internet_Tipo",
        values = "Periodo",
        columns = "Periodo",
        aggregate_function = "count"
    )
)

display(altas)

Pero para poder usarlo con altair tuve que modificar la salida:

In [ ]:
altas2 = \
(
    df
    .filter(
        pl.col("Estado") == "ALTA"
    )
    .group_by(
        "Internet_Tipo",
        "Periodo",
        maintain_order = True
    )
    .agg(
        count=(
            pl.count("Internet_Tipo")
        )
    )
)

display(altas2)

##### Visualización

In [ ]:
altas2_chart = alt.Chart(altas2).mark_bar().encode(
    x=alt.X(
        "Internet_Tipo",
        title=""
    ),
    y=alt.Y(
        "count:Q",
        title="Count"
    ),
    column="Periodo:N",
    color=alt.Color(
        "Periodo:N",
        legend=None
    )
).properties(
    title='Altas de Servicio por Año',
    width=120,
    height=250
)

display(altas2_chart)

Un dato curioso pero obvio, es el salto de contratación de servicio de internet en el primer año de pandemia (2020) en el servicio de aire, casi duplicado al año anterior, y reducido pero por crecimiento del emergente servicio de fibra óptica para el mismo año. Para los siguientes 2 años el servicio de Aire se observa un estancamiento, mientras el servicio de Fibra se triplicó. Y en 2003 el servicio se aire se triplicó, mientras que el de fibra se multiplicó por 6 veces el mismo valor de 2020.

##### -> Bajas del servicio de internet SIPA-AIRE vs SIPA-FIBRA en los últimos 6 años:

Como en la pregunta anterior pero enfocada a las bajas, repito la fórmula para saber cuantos clientes de cada tipo de servicio se dieron de baja por año...

In [ ]:
bajas = \
(
    df
    .filter(
        pl.col("Estado") == "BAJA"
    )
    .pivot(
        index = "Internet_Tipo",
        values = "Periodo",
        columns = "Periodo",
        aggregate_function = "count"
    )
    .fill_null(
        value=0
    )
)

display(bajas)

Al igual que en el caso de las altas, para poder usarlo con altair tuve que modificar la salida:

In [ ]:
bajas2 = \
(
    df
    .filter(
        pl.col("Estado") == "BAJA"
    )
    .group_by(
        "Internet_Tipo",
        "Periodo",
        maintain_order = True
    )
    .agg(
        count=(
            pl.count("Internet_Tipo")
        )
    )
)

display(bajas2)

##### Visualización

In [ ]:
bajas2_chart = alt.Chart(bajas2).mark_bar().encode(
    x=alt.X(
        "Internet_Tipo",
        title=""
    ),
    y=alt.Y(
        "count:Q",
        title="Count"
    ),
    column="Periodo:N",
    color=alt.Color(
        "Periodo:N",
        legend=None
    )
).properties(
    title='Bajas de Servicio por Año',
    width=120,
    height=250
)

display(bajas2_chart)

En los años 2018 y 2019 no hay datos de bajas de fibra óptica. Casi en proporción a las altas en primer y segundo año de pandemia (2020-2021), se observa casi duplicadas las bajas del servicio de aire, si analizamos la situación económica del país en esas fechas puede deberse al factor económico y al mayor consumo del servicio de forma familiar y descontento del rendimiento, en su mayor proporción debido a que el uso normal familiar se vió superado a lo que cada cliente tenía contratado y la creencia de que el servicio les funcionaba mal por problemas del propio proveedor y no la realidad del propio consumo interno.

##### -> Actualizaciones del servicio de internet SIPA AIRE a SIPA-FIBRA según el tipo de internet:

In [ ]:
upgrade_fibra = \
(
    df
    .filter(
        (
            pl.col("Estado") == "UPGRADE"
        ) &
        (
            pl.col("Internet_Tipo") == "Fibra"
        )
    )
    .pivot(
        index = "Internet_Tipo",
        values = "Periodo",
        columns = "Periodo",
        aggregate_function = "count"
    )
)

display(upgrade_fibra)

Dato curioso, solo figuran 2, esto se debe a que estaban reemplazando el contrato anterior por el servicio de fibra en lugar de hacer uno nuevo. Por lo cual no se realizará la visualización.

##### -> Actualizaciones del servicio de internet SIPA AIRE al mismo tipo de servicio según el tipo de internet:

In [ ]:
upgrade_mismo = \
(
    df
    .filter(
        (
            pl.col("Estado") == "UPGRADE"
        ) &
        (
            pl.col("Internet_Tipo") == "Aire"
        )
    )
    .pivot(
        index = "Internet_Tipo",
        values = "Periodo",
        columns = "Periodo",
        aggregate_function = "count"
    )
)

display(upgrade_mismo)

Reformulo el código para utilizarlo en altair:

In [ ]:
upgrade_mismo2 = (
    df
    .filter(
        (
            (
                pl.col("Estado") == "UPGRADE"
            ) &
            (
                pl.col("Internet_Tipo") == "Aire"
            )
        )
    )
    .group_by(
        "Periodo",
        maintain_order = True
    )
    .agg(
        pl
        .count("Periodo")
        .alias("count")
    )
)

display(upgrade_mismo2)

##### Visualización

In [ ]:
upgrade_mismo2_chart = alt.Chart(upgrade_mismo2).mark_bar().encode(
    x=alt.X('Periodo:O', title='Periodo'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('Periodo', legend=None)
).properties(
    title='Upgrades de Servicio por Año',
    width=400,
    height=250
)

display(upgrade_mismo2_chart)

Las actualizaciones tambien son bajas, esto se debe a que en su mayoría los planes chicos que quedaban obsoletos fueron actualizados en su ancho de banda, por ejemplo los planes de 6mbps fueron subidos a 10mbps, de 10mbps a 15mbps, y de 15mbps a 25mbps. Hoy día el plan comercial quedó fijo en 50mbps en servicio por aire, un pqueño grupo manteniene su serivicio en 25mbps y los Upgrades son pocos pero con proyeccion de crecimiento en el tiempo.

##### -> Ver que barrio es el que tiene mayor demanda en el tiempo y según el tipo de internet:

In [ ]:
barrios_max = \
(
    df
    .filter(
        (
            pl.col("Estado") == "ALTA"
        ) |
        (
            pl.col("Estado") == "UPGRADE"
        )
    )
    .group_by(
        ["Periodo", "Internet_Tipo"],
        maintain_order = True
    )
    .agg(
        pl
        .col("Barrio")
        .explode()
        .value_counts(sort = True)
        .limit(1)
        .alias('Barrio_Max')
    )
)

barrios_max2 = \
barrios_max.select(
    pl.all().exclude('Barrio_Max'),
    pl.col('Barrio_Max').list.explode()
).unnest('Barrio_Max')

display(barrios_max2)

En este cuadro se ve un gran crecimiento del servicio por aire en Montecristo en los cuatro años, ya que fue una apertura de disponibilidad de servicio en esa nueva zona, y en el caso de fibra se da lo mismo ya que Housing de Manantiales fue el primer barrio donde se empezó a ofrecer el servicio.
En el primer año de pandemia se observa como lider al barrio Cañuelas, nueva plaza para este tipo de servicio con clientes pasandose del servicio por aire a fibra óptiza.
En 2022 y 2023 se nota el desplazamiento del primer puesto por el barrio GreenVille 2, nueva plaza con la ampliación de cobertura por aire.
En el 2021 cambia a Claros del Bosque por la misma razón, este barrio ahora es la nueva plaza en servicio por fibra óptica manteniendose como lider hasta la fecha.
Para resumir, en todos los casos los grandes crecimientos en altas están dadas por nueva covertura de servicio en dichas zonas.

##### Visualización

In [ ]:
barrios_chart = alt.Chart(barrios_max2).mark_point().encode(
    x=alt.X("Periodo:O", bin=False),
    y=alt.Y("Barrio:N", bin=False),
    color=("Internet_Tipo"),
).properties(
    width=400,
    height=250
)

display(barrios_chart)

In [ ]:
barrios_chart2_temp = df.to_pandas()

filtered_df2 = barrios_chart2_temp[
    (
        (barrios_chart2_temp['Estado'] == 'ALTA') |
        (barrios_chart2_temp['Estado'] == 'UPGRADE')
    ) &
    (barrios_chart2_temp['Internet_Tipo'] == 'Aire')
]

grouped_df2 = filtered_df2.groupby('Periodo')['Barrio'].value_counts().reset_index(name='barrio_unique2')

grouped_df2['Periodo'], grouped_df2['Barrio'] = grouped_df2['Barrio'], grouped_df2['Periodo']

grouped_df_pl2 = pl.DataFrame(grouped_df2)

position_list2 = list(filtered_df2['Barrio'].unique())
position_list2.append(None)
position_dropdown2 = alt.binding_select(options=position_list2, name='Selecciona un Barrio: ')
position_selection2 = alt.selection_point(fields=['Barrio'], bind=position_dropdown2)
color2 = alt.condition(
    position_selection2,
    alt.Color('Periodo:N').legend(None),
    alt.value('lightgray')
)

barrios_chart2 = alt.Chart(grouped_df_pl2).mark_line().encode(
    x=alt.X('Barrio:N', title='Barrios SIPA - Servicio por Aire'),
    y=alt.Y('barrio_unique2:Q', title='Altas & Upgrades'),
    color=color2,
    tooltip=['Periodo:N', 'barrio_unique2:Q']
).properties(
    width=600,
    height=600
).add_params(
    position_selection2
)

display(barrios_chart2)

In [ ]:
barrios_chart3_temp = df.to_pandas()

filtered_df3 = barrios_chart3_temp[
    (
        (barrios_chart3_temp['Estado'] == 'ALTA') |
        (barrios_chart3_temp['Estado'] == 'UPGRADE')
    ) &
    (barrios_chart3_temp['Internet_Tipo'] == 'Fibra')
]

grouped_df3 = filtered_df3.groupby('Periodo')['Barrio'].value_counts().reset_index(name='barrio_unique3')

grouped_df3['Periodo'], grouped_df3['Barrio'] = grouped_df3['Barrio'], grouped_df3['Periodo']

grouped_df_pl3 = pl.DataFrame(grouped_df3)

position_list3 = filtered_df3['Barrio'].unique().tolist()
position_list3.append(None)

position_dropdown3 = alt.binding_select(options=position_list3, name='Selecciona un Barrio: ')
position_selection3 = alt.selection_point(fields=['Barrio'], bind=position_dropdown3)
color3 = alt.condition(
    position_selection3,
    alt.Color('Periodo:N').legend(None),
    alt.value('lightgray'),
)

barrios_chart3 = alt.Chart(grouped_df_pl3).mark_line().encode(
    x=alt.X('Barrio:N', title='Barrios SIPA - Servicio por Fibra'),
    y=alt.Y('barrio_unique3:Q', title='Altas & Upgrades'),
    color=color3,
    tooltip=['Periodo:N', 'barrio_unique3:Q']
).properties(
    width=600,
    height=600
).add_params(
    position_selection3
)

display(barrios_chart3)

"""display(grouped_df3.to_dict())
display(filtered_df3)
display(position_list3)
display(position_selection3)"""

##### -> Ver que ancho de banda es el que tiene mayor demanda en el tiempo y según el tipo de internet:

In [ ]:
bw_max = \
(
    df
    .filter(
        (
            pl.col("Estado") == "ALTA"
        ) |
        (
            pl.col("Estado") == "UPGRADE"
        ) |
        (
            pl.col("Estado") == "FAILUPGRADE"
        )
    )
    .group_by(
        ["Periodo", "Internet_Tipo"],
        maintain_order = True
    )
    .agg(
        pl.col("Ancho_Banda")
            .explode()
            .value_counts(sort = True)
            .limit(1)
            .alias('Ancho_Banda_Max')
    )
)

bw_max2 = \
bw_max.select(
    pl.all().exclude('Ancho_Banda_Max'),
    pl.col('Ancho_Banda_Max').list.explode()
).unnest('Ancho_Banda_Max')

display(bw_max2)

Hasta el año 2020 en pandemia han liderado los planes más chicos (en ambos tipos de servicio), en el caso del servicio por Aire en 2021 se nota ese cambio de opción en el cliente por un servicio de mayor ancho de banda y se mantiene hasta la fecha porque no hay plan mayor a 50mbps por Aire, en cambio en fibra se mantuvo hasta recien llegado 2023 donde ya los clientes aspiran en su mayoría contar el el servicio de 300mbps, el mayor hasta la fecha.

##### Visualización

In [ ]:
bw_chart = bw_max2.to_pandas()
alt.Chart(bw_chart).mark_line().encode(
    x=alt.X("Periodo:O", bin=False, axis=alt.Axis()),
    y=alt.Y("Ancho_Banda:N", scale=alt.Scale(reverse=True)),
    color=("Internet_Tipo"),
).properties(
    width=400,
    height=250
)

##### -> Ver que plan de internet es el que tiene mayor demanda en el tiempo y según el tipo de internet:

In [ ]:
plan_max_temp = \
(
    df
    .filter(
        (
            pl.col("Estado") == "ALTA"
        ) |
        (
            pl.col("Estado") == "UPGRADE"
        ) |
        (
            pl.col("Estado") == "FAILUPGRADE"
        )
    )
    .group_by(
        ["Periodo", "Internet_Tipo"],
        maintain_order = True
    )
    .agg(
        pl
        .col("Plan_SIPA")
        .explode()
        .value_counts(sort = True)
        .limit(1)
        .alias('Plan_SIPA_Max')
    )
)

plan_max = \
plan_max_temp.select(
    pl.all().exclude('Plan_SIPA_Max'),
    pl.col('Plan_SIPA_Max').list.explode()
).unnest('Plan_SIPA_Max')

display(plan_max)

En esta parte se notan los clásicos planes de cada servicio, quedando en menor lugar los planes de IPTV, la mayoría de los clientes prefieren únicamente el servicio de internet, una clara tendencia que el servicio tradicional de TV está en declive...

##### Visualización

In [ ]:
plan_max_chart = alt.Chart(plan_max).mark_line().encode(
    x=alt.X(
        "Periodo:O",
        bin=False
    ),
    y=alt.Y(
        "Plan_SIPA:N",
        scale=alt.Scale(reverse=True)
    ),
    color=(
        "Internet_Tipo"
    ),
).properties(
    width=400,
    height=250
)

display(plan_max_chart)

En la gráfica anterior se ve claramente planchados los valores de los planes dominantes de cada tipo de servicio en los últimos 6 años.

A continuación voy a buscar la comparativa de todos los planes en el mismo tiempo y por año, para lo cual hay que modificar la fórmula:

In [ ]:
import polars as pl

plan_max2 = \
(
    df
    .filter(
        (
            pl.col("Estado") == "ALTA"
        ) |
        (
            pl.col("Estado") == "UPGRADE"
        ) |
        (
            pl.col("Estado") == "FAILUPGRADE"
        )
    )
    .group_by(
        ["Periodo"],
        maintain_order = True
    )
    .agg(
        plan_sipa_unique = pl.col("Plan_SIPA").value_counts(sort = True)
    )
)

display(plan_max2)

In [ ]:
plan_max_chart2_temp = df.to_pandas()

filtered_df = plan_max_chart2_temp[
    (plan_max_chart2_temp['Estado'] == 'ALTA') |
    (plan_max_chart2_temp['Estado'] == 'UPGRADE') |
    (plan_max_chart2_temp['Estado'] == 'FAILUPGRADE')
]

grouped_df = filtered_df.groupby('Periodo')['Plan_SIPA'].value_counts().reset_index(name='plan_sipa_unique')

grouped_df_pl = pl.DataFrame(grouped_df)

plan_max_chart2 = alt.Chart(grouped_df_pl).mark_bar().encode(
    x=alt.X('Plan_SIPA:N', title='Plan SIPA'),
    y=alt.Y('plan_sipa_unique:Q', title='Frequency'),
    color='Periodo:N',
    tooltip=['Plan_SIPA:N', 'plan_sipa_unique:Q']
).properties(
    width=600,
    height=400
)

display(plan_max_chart2)

In [ ]:
plan_max_chart3_temp = df.to_pandas()

filtered_df = plan_max_chart3_temp[
    (plan_max_chart3_temp['Estado'] == 'ALTA') |
    (plan_max_chart3_temp['Estado'] == 'UPGRADE') |
    (plan_max_chart3_temp['Estado'] == 'FAILUPGRADE')
]

grouped_df = filtered_df.groupby('Periodo')['Plan_SIPA'].value_counts().reset_index(name='plan_sipa_unique')

grouped_df['Periodo'], grouped_df['Plan_SIPA'] = grouped_df['Plan_SIPA'], grouped_df['Periodo']

grouped_df_pl = pl.DataFrame(grouped_df)

plan_max_chart3 = alt.Chart(grouped_df_pl).mark_line().encode(
    x=alt.X('Plan_SIPA:N', title='Planes SIPA'),
    y=alt.Y('plan_sipa_unique:Q', title='Frequency'),
    color='Periodo:N',
    tooltip=['Periodo:N', 'plan_sipa_unique:Q']
).properties(
    width=600,
    height=400
)

display(plan_max_chart3)

##### -> Ver que cantidad de clientes tienen contratado IPTV en el tiempo:

In [ ]:
iptv = \
(
    df
    .filter
    (
        (
            (
                pl.col("Estado") == "ALTA"
            ) |
            (
                pl.col("Estado") == "UPGRADE"
            )
        ) &
        (
            pl.col("IPTV") == 1
        )
    )
    .pivot(
        index = "IPTV",
        values = "Periodo",
        columns = "Periodo",
        aggregate_function = "count"
    )
)

display(iptv)

Si bien el servicio de TV tradicional está en declive a nivel mundial, se nota que los clientes, en menor medida, lo han adquirido a un ritmo de crecimiento del 100% anual desde el año 2021.

Por otro lado esta fórmula me dio muchos problemas para hacerla funcionar con altair, asi que la reformulé a:

In [ ]:
iptv2 = (
    df
    .filter(
        (
            (
                pl.col("Estado") == "ALTA"
            ) |
            (
                pl.col("Estado") == "UPGRADE"
            )
        ) &
        (
            pl.col("IPTV") == 1
        )
    )
    .group_by(
        "Periodo",
        maintain_order = True
    )
    .agg(
        pl
        .count("Periodo")
        .alias("count")
    )
)

display(iptv2)

##### Visualización

In [ ]:
iptv2_chart = alt.Chart(iptv2).mark_bar().encode(
    x=alt.X('Periodo:O', title='Periodo'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('Periodo', legend=None)
).properties(
    title='IPTV Contratados por Año',
    width=400,
    height=250
)

display(iptv2_chart)

Y aquí ya puede verse visualmente el crecimiento del servicio de IPTV en este proveedor.

-----.-----


## Implementación de Machine Learning

### Instalar librerías

In [ ]:
!python -m pip uninstall scikit-learn

In [ ]:
!python -m pip install scikit-learn

### Importar librerías

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, recall_score
from joblib import dump, load

### Cargar y Verificar el Dataset

In [ ]:
df = pl.read_csv(file_path_cleaned)

### Agregar columna para predecir la -Renovación del Cliente-

In [ ]:
random_values = np.random.randint(2, size=len(df))

df = df.with_columns([
    pl.lit(random_values).alias("cliente_renueva")
])

In [ ]:
df.sample(3)

### La empresa desea saber que clientes pueden renovar el servicio

In [ ]:
df = df.drop(['ID_Cliente', 'ID_Contrato'])

In [ ]:
dfp = df.to_pandas()

In [ ]:
cliente_renueva = dfp['cliente_renueva'].copy()

dfp = dfp.drop('cliente_renueva', axis=1)

In [ ]:
dfp.sample(3)

In [ ]:
cliente_renueva

In [ ]:
original_count = len(dfp)
training_size = 0.50
test_size = (1 - training_size) / 2

training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print(training_count, test_count, validation_count, original_count)

In [ ]:
df.sample(3)

In [ ]:
train_x, rest_x, train_y, rest_y = train_test_split(dfp, cliente_renueva, train_size=training_count)
test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_count)

print(len(train_x), len(test_x), len(validate_x))

In [ ]:
one_hot_encoding = ColumnTransformer([
    (
        'one_hot_encode',
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        [
            "Barrio",
            "Periodo",
            'Plan_SIPA',
            "Internet_Tipo",
            "Estado"
        ]
    )
])

In [ ]:
scaler = ColumnTransformer([
    ("scaler", RobustScaler(), ["Ancho_Banda"])
])

In [ ]:
model_pipeline = pipe = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("categorical", one_hot_encoding),
                    ("scaled", scaler)
                ]
            ),
        )
    ]
)

In [ ]:
binarized = model_pipeline.fit_transform(train_x)
binarized.shape

In [ ]:
binarized

In [ ]:
training_pipeline = Pipeline([
    ("model", clone(model_pipeline)),
    ("estimators", RandomForestClassifier(n_estimators=150))
])

In [ ]:
training_dataset = pd.concat([train_x, validate_x])
training_response = pd.concat([train_y, validate_y])

In [ ]:
training_pipeline.fit(training_dataset, training_response)

In [ ]:
ml_pred_y = training_pipeline.predict(test_x)

print(accuracy_score(ml_pred_y, test_y))
print(recall_score(ml_pred_y, test_y))

In [ ]:
dump(training_pipeline, "model_pipeline.joblib")

In [ ]:
new_customers = pd.read_csv(file_path_cleaned)

In [ ]:
new_customers.head(2)

In [ ]:
training_model = load("model_pipeline.joblib")

In [ ]:
new_customers['cliente_renueva'] = training_model.predict(new_customers)

In [ ]:
len(new_customers[new_customers.cliente_renueva == 0]), len(new_customers[new_customers.cliente_renueva == 1])

In [ ]:
new_customers[['ID_Cliente', 'ID_Contrato', 'cliente_renueva']].sort_values(by='cliente_renueva', ascending=False)

## Implementación del Deep Learning

### Instalar librerías

In [ ]:
!python -m pip uninstall keras tensorflow

In [ ]:
!python -m pip install keras tensorflow

### Importar librerías

In [ ]:
import keras

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

In [ ]:
classifier = Sequential()

### Creando Neuronas para automatizar la predicción ML

In [ ]:
classifier.add(Dense(units=2, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(units=2, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units=2, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
(train_x, train_y)

In [ ]:
# TODO: Pendiente verificar tipo
classifier.fit(train_x, train_y, epochs = 100)

In [ ]:
dl_pred_y = classifier.predict(test_x)
dl_pred_y = (dl_pred_y > 0.5)

In [ ]:
# TODO: Pendiente calcular la matris
#cm = confusion_matrix(y_test, y_pred)
#cm